In [1]:
import os
os.chdir(r"c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination")
import re
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import optuna
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support, classification_report, accuracy_score, recall_score, precision_score, f1_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, cross_val_predict
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from unidecode import unidecode
from xgboost.sklearn import XGBRegressor

In [2]:
#warnings.simplefilter("ignore")

In [3]:
df = pd.read_csv("data/cleaned_data/bulletins_w_labels_and_content.csv", dtype={'ID': object})  
df = df[["ID", "Job Description", "Apps Received", "Female", "Male", "Unknown_Gender", "Cleaned text"]]
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,Cleaned text
0,9206,311 DIRECTOR,54,20,31,3,director class code open date annual salary du...
1,1223,ACCOUNTING CLERK,648,488,152,8,accounting clerk class code open date exam ope...
2,7260,AIRPORT MANAGER,51,13,37,1,airport manager class code open date exam open...
3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,airport police lieutenant class code open date...
4,2400,AQUARIST,40,15,24,1,aquarist class code open date annual salary ca...
...,...,...,...,...,...,...,...
172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,wastewater treatment laboratory manager class ...
173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,wastewater treatment operator class code open ...
174,7857,WATER MICROBIOLOGIST,179,89,82,8,water microbiologist class code open date revi...
175,3912,WATER UTILITY WORKER,96,2,92,2,water utility worker class code open date exam...


In [4]:
df["Apps Received (unknown gender removed)"] = df["Male"] + df["Female"]
df["Male share"] = round(df["Male"] / df["Apps Received (unknown gender removed)"], 3)
df["Female share"] = round(df["Female"] / df["Apps Received (unknown gender removed)"], 3)
df["Male share (unknown gender included)"] =  round(df["Male"] / df["Apps Received"], 3)
df["Female share (unknown gender included)"] =  round(df["Female"] / df["Apps Received"], 3)
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,Cleaned text,Apps Received (unknown gender removed),Male share,Female share,Male share (unknown gender included),Female share (unknown gender included)
0,9206,311 DIRECTOR,54,20,31,3,director class code open date annual salary du...,51,0.608,0.392,0.574,0.370
1,1223,ACCOUNTING CLERK,648,488,152,8,accounting clerk class code open date exam ope...,640,0.238,0.762,0.235,0.753
2,7260,AIRPORT MANAGER,51,13,37,1,airport manager class code open date exam open...,50,0.740,0.260,0.725,0.255
3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,airport police lieutenant class code open date...,47,0.809,0.191,0.792,0.188
4,2400,AQUARIST,40,15,24,1,aquarist class code open date annual salary ca...,39,0.615,0.385,0.600,0.375
...,...,...,...,...,...,...,...,...,...,...,...,...
172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,wastewater treatment laboratory manager class ...,15,0.600,0.400,0.562,0.375
173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,wastewater treatment operator class code open ...,122,0.926,0.074,0.904,0.072
174,7857,WATER MICROBIOLOGIST,179,89,82,8,water microbiologist class code open date revi...,171,0.480,0.520,0.458,0.497
175,3912,WATER UTILITY WORKER,96,2,92,2,water utility worker class code open date exam...,94,0.979,0.021,0.958,0.021


In [5]:
df = df.sample(frac=1).reset_index(drop=True)
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,Cleaned text,Apps Received (unknown gender removed),Male share,Female share,Male share (unknown gender included),Female share (unknown gender included)
0,5235,SENIOR LOAD DISPATCHER,18,2,14,2,senior load dispatcher class code open date ex...,16,0.875,0.125,0.778,0.111
1,2495,VOLUNTEER COORDINATOR,326,227,94,5,volunteer coordinator class code open date exa...,321,0.293,0.707,0.288,0.696
2,3775,SHEET METAL WORKER,271,3,265,3,sheet metal worker class code open date exam o...,268,0.989,0.011,0.978,0.011
3,3421,TRAFFIC PAINTER AND SIGN POSTER,1225,61,1150,14,traffic painter sign poster class code open da...,1211,0.950,0.050,0.939,0.050
4,1861,UTILITY BUYER,126,64,58,4,utility buyer class code open date exam open c...,122,0.475,0.525,0.460,0.508
...,...,...,...,...,...,...,...,...,...,...,...,...
172,4240,FIRE SPRINKLER INSPECTOR,83,3,79,1,fire sprinkler inspector class code open date ...,82,0.963,0.037,0.952,0.036
173,3802,COMMUNICATIONS CABLE WORKER,393,2,386,5,communication cable worker class code open dat...,388,0.995,0.005,0.982,0.005
174,2472,SUPERINTENDENT OF RECREATION AND PARKS OPERATIONS,26,8,18,0,superintendent recreation park operation class...,26,0.692,0.308,0.692,0.308
175,5887,WATER TREATMENT SUPERVISOR,16,0,15,1,water treatment supervisor class code open dat...,15,1.000,0.000,0.938,0.000


In [6]:
X = df["Cleaned text"]
y = df["Male share"]

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X).toarray()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1001)

In [9]:
lin_reg = LinearRegression()

In [10]:
lin_reg.fit(X_train, y_train)

LinearRegression()

In [11]:
score = lin_reg.score(X_test, y_test)

In [12]:
score

0.3029337019999674

In [13]:
scores = cross_val_score(lin_reg, X, y, cv=10)

In [14]:
scores

array([0.66522046, 0.30128712, 0.12389617, 0.63643065, 0.56745595,
       0.1734767 , 0.19236536, 0.54139226, 0.56984163, 0.26709787])

In [15]:
def objective(trial):

    # classifier_name = trial.suggest_categorical("classifier", ["LinReg", "RandomForest", "DecTree", "SVR", "GradientBoost", "ElasticNet", "SGD", "BayesRidge", "CatBoost", "KernelRidge", "XGBoost", "LGBM"])
    classifier_name = trial.suggest_categorical("classifier", ["RandomForest", "GradientBoost", "CatBoost", "LGBM"])
    
    # Step 2. Setup values for the hyperparameters:
    # if classifier_name == 'LinReg':
    #     classifier_obj = LinearRegression()
    if classifier_name == "RandomForest":
    #else:
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = RandomForestRegressor(n_estimators=rf_n_estimators, max_depth=rf_max_depth)
    # elif classifier_name == "DecTree":
    #     classifier_obj = DecisionTreeRegressor()
    # elif classifier_name == "SVR":
    #     #svr_c = trial.suggest_float('svr_c', 1e-10, 1e10, log=True)
    #     classifier_obj = SVR(gamma='auto')
    elif classifier_name == "GradientBoost":
        gb_n_estimators = trial.suggest_int("gb_n_estimators", 10, 1000)
        gb_lr = trial.suggest_float("gb_lr", 1e-10, 1e10, log=True)
        classifier_obj = GradientBoostingRegressor(learning_rate=gb_lr, n_estimators=gb_n_estimators)
    # elif classifier_name == "ElasticNet":
    #     classifier_obj = ElasticNet()
    # elif classifier_name == "SGD":
    #     classifier_obj = SGDRegressor()
    # elif classifier_name == "BayesRidge":
    #     classifier_obj = BayesianRidge()
    elif classifier_name == "CatBoost":
        #cb_lr = trial.suggest_float("cb_lr", 1e-10, 1e10, log=True)
        classifier_obj = CatBoostRegressor()
    # elif classifier_name == "KernelRidge":
    #     classifier_obj = KernelRidge()
    # elif classifier_name == "XGBoost":
    #     classifier_obj = XGBRegressor()
    elif classifier_name == "LGBM":
        lgbm_n_estimators = trial.suggest_int("lgbm_n_estimators", 10, 1000)
        lgbm_max_depth = trial.suggest_int("lgbm_max_depth", 2, 32, log=True)
        classifier_obj = LGBMRegressor(n_estimators=lgbm_n_estimators, max_depth=lgbm_max_depth)

    # Step 3: Scoring method:
    score = cross_val_score(classifier_obj, X, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2023-01-09 15:45:31,497] A new study created in memory with name: no-name-7e98c05a-e7f5-49db-bb41-7ff0d89b8042
[W 2023-01-09 15:45:38,048] Trial 0 failed because of the following error: The value nan is not acceptable.
[I 2023-01-09 15:46:07,108] Trial 1 finished with value: 0.5307620110314387 and parameters: {'classifier': 'CatBoost'}. Best is trial 1 with value: 0.5307620110314387.
[I 2023-01-09 15:46:32,768] Trial 2 finished with value: 0.5307620110314387 and parameters: {'classifier': 'CatBoost'}. Best is trial 1 with value: 0.5307620110314387.
[I 2023-01-09 15:46:39,328] Trial 3 finished with value: 0.5098793832609321 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 324, 'rf_max_depth': 19}. Best is trial 1 with value: 0.5307620110314387.
[I 2023-01-09 15:46:40,749] Trial 4 finished with value: 0.5351803772256639 and parameters: {'classifier': 'LGBM', 'lgbm_n_estimators': 78, 'lgbm_max_depth': 32}. Best is trial 4 with value: 0.5351803772256639.
[I 2023-01-09 1

KeyboardInterrupt: 

In [34]:
def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_float("learning_rate", 0.001, 0.02, step=0.001)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_float('l2_leaf_reg', 1.0, 5.5, step=0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    #param['iterations'] = 10000
    #param['use_best_model'] = True
    param['eval_metric'] = 'RMSE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = 1
    param['logging_level'] = 'Silent'
    
    regressor = CatBoostRegressor(**param)

    regressor.fit(X_train, y_train, early_stopping_rounds=100)
    loss = mean_squared_error(y_test, regressor.predict(X_test))
    return loss
    

In [35]:
study = optuna.create_study(study_name=f'catboost-seed{1}')
study.optimize(objective, n_trials=200, timeout=24000)

[I 2023-01-10 08:58:28,874] A new study created in memory with name: catboost-seed1
[I 2023-01-10 09:00:06,943] Trial 0 finished with value: 0.031040337805151554 and parameters: {'learning_rate': 0.005, 'depth': 9, 'l2_leaf_reg': 5.5, 'min_child_samples': 1}. Best is trial 0 with value: 0.031040337805151554.
[I 2023-01-10 09:01:28,512] Trial 1 finished with value: 0.040813299781956615 and parameters: {'learning_rate': 0.001, 'depth': 15, 'l2_leaf_reg': 1.5, 'min_child_samples': 8}. Best is trial 0 with value: 0.031040337805151554.
[I 2023-01-10 09:02:39,974] Trial 2 finished with value: 0.03245324351548526 and parameters: {'learning_rate': 0.002, 'depth': 13, 'l2_leaf_reg': 1.0, 'min_child_samples': 8}. Best is trial 0 with value: 0.031040337805151554.
[I 2023-01-10 09:03:11,248] Trial 3 finished with value: 0.02709879194730593 and parameters: {'learning_rate': 0.007, 'depth': 11, 'l2_leaf_reg': 1.5, 'min_child_samples': 32}. Best is trial 3 with value: 0.02709879194730593.
[I 2023-01-

KeyboardInterrupt: 

In [27]:
study.best_params

{'learning_rate': 0.009000000000000001,
 'depth': 15,
 'l2_leaf_reg': 3.0,
 'min_child_samples': 32}

In [30]:
param = {}
param['learning_rate'] = 0.009
param['depth'] = 15
param['l2_leaf_reg'] = 3.0
param['min_child_samples'] = 32
param['grow_policy'] = 'Depthwise'
#param['iterations'] = 10000
param['eval_metric'] = 'RMSE'
param['od_type'] = 'Iter'
param['od_wait'] = 20
param['random_state'] = 1
param['logging_level'] = 'Silent'
    
regressor = CatBoostRegressor(**param)

score = cross_val_score(regressor, X, y, n_jobs=-1, cv=3)
accuracy = score.mean()

In [31]:
accuracy

0.5738416188135632

In [32]:
regressor.fit(X_train, y_train, early_stopping_rounds=100)
loss = mean_squared_error(y_test, regressor.predict(X_test))

In [33]:
loss

0.0262218829760942